# Chapter 3: Classification

Chapter ini membahas salah satu tugas paling fundamental dalam Machine Learning, yaitu **classification**. Berbeda dengan regresi yang memprediksi nilai kontinu, classification bertujuan memprediksi **kelas atau label diskrit**.

Classification banyak digunakan dalam berbagai aplikasi nyata, seperti deteksi spam, pengenalan wajah, diagnosis medis, dan pengenalan karakter tulisan tangan. Pada chapter ini, konsep classification akan dibahas secara bertahap menggunakan dataset MNIST sebagai studi kasus utama.

Tujuan utama chapter ini adalah membantu pembaca memahami:
- bagaimana classifier bekerja,
- bagaimana mengevaluasi performa classifier dengan benar,
- serta bagaimana menangani berbagai jenis masalah classification.

## 1. What Is Classification?

Classification adalah jenis supervised learning di mana model mempelajari hubungan antara fitur input dan **label kategorikal**. Label ini merepresentasikan kelas tertentu yang telah ditentukan sebelumnya.

Tidak seperti regresi yang menghasilkan angka kontinu, hasil dari classification berupa keputusan diskrit, misalnya "ya/tidak" atau pilihan kelas tertentu.

Beberapa contoh masalah classification antara lain:
- email spam atau bukan spam,
- transaksi penipuan atau normal,
- pengenalan digit tulisan tangan,
- klasifikasi sentimen teks.

Pada praktiknya, tantangan utama classification bukan hanya membangun model, tetapi juga mengevaluasi dan memahami jenis kesalahan yang dilakukan model.

## 2. The MNIST Dataset

MNIST adalah dataset klasik dalam Machine Learning yang berisi **70.000 gambar digit tulisan tangan** dari angka 0 hingga 9. Setiap gambar memiliki ukuran 28 × 28 piksel dan direpresentasikan sebagai vektor berdimensi 784.

![](Figure/figure3.1.png)
Dataset ini sering digunakan sebagai dataset pembelajaran karena:
- strukturnya sederhana dan mudah dipahami,
- cukup menantang untuk menunjukkan kesalahan classification,
- serta menjadi benchmark bagi banyak algoritma.

Meskipun terlihat sederhana, MNIST mengandung berbagai variasi tulisan tangan yang membuat tugas classification menjadi tidak trivial.

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

Dataset MNIST dimuat dalam bentuk dictionary-like object. Dua komponen terpenting yang akan digunakan adalah:
- `data`: fitur input berupa nilai piksel
- `target`: label digit

Langkah berikutnya adalah memisahkan fitur dan label.

In [ ]:
X, y = mnist["data"], mnist["target"]
X.shape, y.shape

Output di atas menunjukkan bahwa dataset terdiri dari 70.000 instance, masing-masing dengan 784 fitur. Setiap fitur merepresentasikan intensitas piksel dari 0 hingga 255.

## 3. Visualizing a Digit

Sebelum melatih model, penting untuk memahami bagaimana data direpresentasikan. Vektor berdimensi 784 dapat diubah kembali menjadi matriks 28 × 28 dan divisualisasikan sebagai gambar.

Visualisasi ini membantu memastikan bahwa data dimuat dengan benar dan memberikan intuisi tentang tingkat variasi dalam tulisan tangan.

In [ ]:
import matplotlib.pyplot as plt

some_digit = X[0]
some_digit_image = some_digit.reshape(28, 28)

plt.imshow(some_digit_image, cmap="binary")
plt.axis("off")
plt.show()

Gambar di atas merupakan contoh digit tulisan tangan dari dataset MNIST. Kita dapat memeriksa label aslinya untuk memastikan kesesuaian antara gambar dan target.

In [ ]:
y[0]

Label pada dataset MNIST disimpan dalam bentuk string. Untuk kompatibilitas dengan algoritma Machine Learning, label ini perlu dikonversi ke tipe numerik.

In [ ]:
import numpy as np
y = y.astype(np.uint8)

## 4. Train–Test Split

Untuk mengevaluasi performa model secara objektif, dataset perlu dibagi menjadi **training set** dan **test set**. Training set digunakan untuk melatih model, sedangkan test set digunakan untuk evaluasi akhir.

Pada dataset MNIST, pembagian data sudah disediakan secara default:
- 60.000 instance pertama sebagai training set
- 10.000 instance terakhir sebagai test set

Pembagian ini memastikan bahwa evaluasi dilakukan pada data yang benar-benar belum pernah dilihat oleh model.

In [ ]:
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

Training set pada MNIST telah diacak (*shuffled*) sebelumnya, sehingga distribusi kelas relatif merata dan cocok untuk digunakan dalam cross-validation.

## 5. Binary Classification: Detecting the Digit 5

Sebagai langkah awal dalam mempelajari classification, kita menyederhanakan masalah menjadi **binary classification**.

Pada contoh ini, model hanya bertugas menentukan apakah sebuah gambar merupakan angka **5** atau **bukan 5**. Pendekatan ini membantu memahami konsep dasar classification sebelum beralih ke multiclass classification.

In [ ]:
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

Label baru ini bernilai `True` jika digit tersebut adalah angka 5, dan `False` jika bukan. Dengan demikian, masalah multiclass diubah menjadi masalah dua kelas.

## 6. Training an SGD Classifier

Untuk membangun classifier pertama, kita menggunakan **Stochastic Gradient Descent (SGD) Classifier**.

SGD merupakan algoritma yang efisien untuk dataset besar karena memperbarui parameter model secara bertahap, satu instance pada satu waktu. Model ini sangat cocok untuk dataset MNIST yang berukuran besar.

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

Model SGDClassifier telah dilatih untuk membedakan antara digit 5 dan digit lainnya. Kita dapat mencoba melakukan prediksi pada satu contoh digit.

In [ ]:
sgd_clf.predict([some_digit])

Jika hasil prediksi adalah `True`, maka model mengklasifikasikan digit tersebut sebagai angka 5.

## 7. Evaluating the Classifier Using Accuracy

Langkah evaluasi paling sederhana adalah menggunakan **accuracy**, yaitu proporsi prediksi yang benar dibandingkan seluruh data.

Namun, pada masalah classification dengan distribusi kelas yang tidak seimbang, accuracy dapat memberikan gambaran yang menyesatkan.

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")

Nilai accuracy yang tinggi terlihat menjanjikan. Namun, karena hanya sekitar 10% data merupakan digit 5, classifier sederhana yang selalu memprediksi "bukan 5" pun dapat mencapai accuracy yang tinggi.

Hal ini menunjukkan bahwa **accuracy saja tidak cukup** untuk mengevaluasi classifier secara menyeluruh. Pada bagian berikutnya, kita akan mempelajari metrik evaluasi yang lebih informatif.

## 8. Confusion Matrix

Accuracy tidak memberikan informasi tentang **jenis kesalahan** yang dilakukan classifier. Untuk itu, digunakan **confusion matrix**, yang menunjukkan jumlah prediksi benar dan salah untuk setiap kelas.

Pada binary classification, confusion matrix terdiri dari empat komponen utama:
- **True Positive (TP)**: digit 5 yang diprediksi sebagai 5
- **True Negative (TN)**: bukan digit 5 yang diprediksi bukan 5
- **False Positive (FP)**: bukan digit 5 tetapi diprediksi sebagai 5
- **False Negative (FN)**: digit 5 tetapi diprediksi bukan 5

Dengan confusion matrix, kita dapat memahami jenis kesalahan mana yang paling sering terjadi.

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)
confusion_matrix(y_train_5, y_train_pred)

Confusion matrix di atas menunjukkan bahwa sebagian besar digit bukan 5 diklasifikasikan dengan benar. Namun, masih terdapat kesalahan berupa *false positives* dan *false negatives*.

Jenis kesalahan ini memiliki konsekuensi berbeda tergantung konteks aplikasi. Oleh karena itu, kita memerlukan metrik evaluasi yang lebih spesifik.

## 9. Precision and Recall

**Precision** mengukur seberapa akurat prediksi positif yang dibuat oleh classifier, sedangkan **recall** mengukur seberapa banyak instance positif yang berhasil dideteksi.

Secara intuitif:
- Precision tinggi berarti sedikit *false positives*
- Recall tinggi berarti sedikit *false negatives*

Kedua metrik ini sering berada dalam trade-off dan perlu dipertimbangkan bersama.

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_5, y_train_pred), recall_score(y_train_5, y_train_pred)

Nilai precision dan recall memberikan gambaran yang lebih realistis tentang performa classifier dibandingkan accuracy, terutama pada dataset yang tidak seimbang.

## 10. The F1 Score

**F1-score** merupakan rata-rata harmonik dari precision dan recall. Metrik ini memberikan satu nilai ringkas yang mempertimbangkan kedua aspek tersebut secara seimbang.

F1-score sangat berguna ketika kita ingin menyeimbangkan antara precision dan recall.

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_train_5, y_train_pred)

Nilai F1-score yang dihasilkan mencerminkan keseimbangan antara kemampuan classifier dalam mendeteksi digit 5 dan ketepatan prediksi tersebut.

## 11. Precision–Recall Trade-off

Sebagian besar classifier tidak langsung menghasilkan label kelas, melainkan **skor keputusan** (*decision score*). Dengan mengubah threshold keputusan, kita dapat mengatur trade-off antara precision dan recall.

Pemilihan threshold sangat bergantung pada tujuan aplikasi dan jenis kesalahan yang paling ingin dihindari.

In [ ]:
y_scores = cross_val_predict(
    sgd_clf, X_train, y_train_5, cv=3,
    method="decision_function"
)

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

Kurva precision–recall menggambarkan bagaimana precision menurun ketika recall meningkat. Grafik ini membantu menentukan titik operasi (*operating point*) yang sesuai dengan kebutuhan sistem.

## 12. The ROC Curve

Selain precision–recall curve, metode evaluasi lain yang umum digunakan adalah **ROC (Receiver Operating Characteristic) curve**.

ROC curve memplot **True Positive Rate (Recall)** terhadap **False Positive Rate** pada berbagai nilai threshold. Kurva ini memberikan sudut pandang berbeda dalam mengevaluasi performa classifier.

Classifier yang baik akan memiliki ROC curve yang mendekati sudut kiri atas grafik.

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

ROC curve sering digunakan untuk membandingkan beberapa classifier. Model dengan kurva yang lebih mendekati sudut kiri atas umumnya memiliki performa yang lebih baik.

## 13. Multiclass Classification

Setelah memahami binary classification, kita kembali ke permasalahan **multiclass classification**, yaitu klasifikasi dengan lebih dari dua kelas.

Dataset MNIST secara alami merupakan masalah multiclass karena memiliki 10 kelas (digit 0 sampai 9).

In [ ]:
sgd_clf.fit(X_train, y_train)
sgd_clf.predict([some_digit])

Secara default, `SGDClassifier` menggunakan strategi **one-vs-all**, yaitu melatih satu classifier untuk setiap kelas dan memilih kelas dengan skor tertinggi.

## 14. Error Analysis

Analisis kesalahan bertujuan untuk memahami **jenis kesalahan spesifik** yang dilakukan oleh classifier.

Dengan menggunakan confusion matrix multiclass, kita dapat melihat pasangan digit mana yang paling sering tertukar, misalnya digit 3 yang sering diklasifikasikan sebagai 5.

In [ ]:
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train, cv=3)
conf_mx = confusion_matrix(y_train, y_train_pred)
conf_mx

Confusion matrix multiclass memungkinkan kita mengidentifikasi pola kesalahan sistematis, yang dapat menjadi dasar untuk perbaikan model atau preprocessing.

## 15. Multilabel Classification

Dalam **multilabel classification**, satu instance dapat memiliki lebih dari satu label secara bersamaan.

Sebagai contoh, sebuah digit dapat diklasifikasikan sebagai:
- digit besar (>= 7)
- digit ganjil

Kedua label ini dapat bernilai benar secara bersamaan.

In [ ]:
y_train_large = (y_train >= 7)
y_train_odd = (y_train % 2 == 1)
y_multilabel = np.c_[y_train_large, y_train_odd]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_multilabel)

Classifier multilabel menghasilkan beberapa prediksi sekaligus untuk satu instance, menunjukkan fleksibilitas pendekatan classification.

## 16. Multioutput Classification

Multioutput classification merupakan generalisasi dari multilabel classification, di mana setiap label dapat memiliki lebih dari dua kemungkinan nilai.

Contoh penerapannya adalah sistem *denoising*, di mana model memprediksi nilai piksel bersih dari input gambar yang mengandung noise.

## Closing Summary (Chapter 3)

Chapter 3 menekankan bahwa classification bukan hanya tentang melatih model, tetapi juga tentang **mengevaluasi dan memahami perilaku classifier secara mendalam**.

Melalui pembahasan confusion matrix, precision–recall trade-off, ROC curve, serta berbagai jenis classification, kita belajar bahwa pemilihan metrik evaluasi sangat bergantung pada konteks permasalahan.

Pemahaman yang kuat terhadap konsep-konsep ini menjadi fondasi penting sebelum mempelajari topik lanjutan seperti ensemble learning dan neural networks pada chapter berikutnya.